# Import modules

In [1]:
import json
import requests
import urllib.parse
import time
from bs4 import BeautifulSoup
import pprint
from extract_hwpx_latest import WebTextExtractor
import nest_asyncio
import asyncio
from pprint import pprint
import asyncio
import re
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import traceback
from crawl4ai import AsyncWebCrawler
from extract_hwpx_latest import WebTextExtractor
import nest_asyncio
import asyncio
import feedparser
import urllib.request
import nest_asyncio
nest_asyncio.apply()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


###  hwpx뷰어에서 텍스트 추출

In [2]:
IFRAME_KEY = "docviewer/skin/doc.html"  # 미리보기 iframe 식별 키

def _extract_urls_from_iframes(soup, base):
    """이미 열려 있는 프리뷰 iframe의 src 추출 (가장 확실/간단)"""
    urls = set()
    for iframe in soup.select("div.bbs_file_preview iframe[src]"):
        src = iframe.get("src")
        if src and IFRAME_KEY in src:
            urls.add(urljoin(base, src))
    return urls

def _find_any_rs_suffix(soup):
    """
    같은 페이지 내 다른 iframe에서 rs 파라미터만 떼어 재사용.
    예: /docviewer/result/ntc0021/49374/1/202509
    """
    any_iframe = soup.select_one(f"div.bbs_file_preview iframe[src*='docviewer/result/']")
    if not any_iframe or not any_iframe.get("src"):
        return None
    m = re.search(r"[?&]rs=([^&]+)", any_iframe["src"])
    return m.group(1) if m else None

def _extract_urls_from_onclick(soup, base):
    """
    onclick="fnConvertDocViewer('brd_id','seq','file_seq')" + 파일명으로
    미리보기 URL 조립. rs는 같은 페이지 iframe에서 재사용.
    """
    urls = set()
    onclick_re = re.compile(r"fnConvertDocViewer\('([^']+)','([^']+)','([^']+)'\)")
    rs_suffix = _find_any_rs_suffix(soup)

    for a in soup.select("a.bbs_icon_preveiw[onclick]"):
        m = onclick_re.search(a.get("onclick", ""))
        if not m:
            continue
        brd_id, seq, file_seq = m.groups()

        li = a.find_parent("li")
        strong = li.select_one(".bbs_file_cont > strong") if li else None
        filename = strong.get_text(strip=True) if strong else None

        if not filename:
            continue

        # rs 추정/조립
        rs = None
        # 1) 같은 li에 프리뷰 iframe이 있으면 그걸 우선 신뢰
        if li:
            ifr = li.select_one("div.bbs_file_preview iframe[src]")
            if ifr and ifr.get("src"):
                m2 = re.search(r"[?&]rs=([^&]+)", ifr["src"])
                if m2:
                    rs = m2.group(1)

        # 2) 페이지 내 임의 iframe의 연월(YYYYMM)만 재사용
        if not rs and rs_suffix:
            # rs_suffix 예: /docviewer/result/ntc0021/49374/1/202509
            tail = rs_suffix.strip("/").split("/")
            yyyy_mm = tail[-1] if tail else ""
            rs = f"/docviewer/result/{brd_id}/{seq}/{file_seq}/{yyyy_mm}"

        if rs:
            url = f"/docviewer/skin/doc.html?fn={filename}&rs={rs}"
            urls.add(urljoin(base, url))

    return urls
def _extract_synapviewer_urls(soup, base, page_url):
    """
    바로보기(synapviewer) 링크 추출
    """
    urls = set()
    synapviewer_re = re.compile(r"window\.open\('(synapviewer\.do[^']+)'\)")
    
    # 페이지 URL에서 경로 추출 (예: /home/)
    from urllib.parse import urlparse
    parsed = urlparse(page_url)
    page_path = parsed.path.rsplit('/', 1)[0] + '/'  # 마지막 파일명 제거
    
    for div in soup.select("div[id^='fileDiv']"):
        links = div.select("a")
        for link in links:
            onclick = link.get("onclick", "")
            m = synapviewer_re.search(onclick)
            if m:
                viewer_url = m.group(1)
                # 상대 경로면 페이지 경로와 결합
                if not viewer_url.startswith(('http://', 'https://', '/')):
                    viewer_url = page_path + viewer_url
                urls.add(urljoin(base, viewer_url))
    
    return urls

# 또는 특정 fileDiv3만 추출하려면:
def _extract_filediv3_synapviewer_url(soup, base):
    """
    fileDiv3의 바로보기 링크만 추출
    """
    synapviewer_re = re.compile(r"window\.open\('(synapviewer\.do[^']+)'\)")
    
    div3 = soup.select_one("#fileDiv3")
    if not div3:
        return None
    
    for link in div3.select("a"):
        onclick = link.get("onclick", "")
        m = synapviewer_re.search(onclick)
        if m:
            return urljoin(base, m.group(1))
    
    return None
async def extract_preview_urls(page_url: str, base_url: str = None):
    """
    주어진 게시글 URL에서 미리보기 URL들(set)을 추출.
    """
    if base_url is None:
        from urllib.parse import urlparse
        p = urlparse(page_url)
        base_url = f"{p.scheme}://{p.netloc}"

    async with AsyncWebCrawler() as crawler:
        r = await crawler.arun(page_url)
        html = r.html

    soup = BeautifulSoup(html, "html.parser")

    urls = set()
    urls |= _extract_urls_from_iframes(soup, base_url)
    urls |= _extract_synapviewer_urls(soup, base_url, page_url)  # page_url 전달
    
    return list(urls)

# hwpx sources

In [ ]:
with open('config.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)


all_articles = {}
for hwpx_source in json_data['hwpx_sources']:
    hwpx_url = hwpx_source['url']
    pagenum = hwpx_source.get('pagenum', 1)
    name =  hwpx_source['name']
    print(f'{hwpx_source['name']}')
    
    if name == '한국소비자원':
        continue
        START_PAGE=1; END_PAGE= pagenum + 1
        articles = []
        for page_num in range(START_PAGE, END_PAGE):
            current_list_url = hwpx_url.format(page_num)
            parsed_url = urllib.parse.urlparse(current_list_url)
            base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
            print(f"Base URL: {base_url}") 
            
            print(f"--- 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            try:
                init_response = requests.get(current_list_url, timeout=10)
                init_response.encoding = 'utf-8'
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                # 뉴스 항목 선택 (테이블 내 tr 태그)
                news_list = soup.select('table.board tbody tr')
                
                if not news_list:
                    print(f"경고: {page_num} 페이지에서 기사를 찾을 수 없습니다.")
                    break

                print(f'완료: {len(news_list)}개 기사 발견')

                # 각 기사 처리
                for i, item in enumerate(news_list):
                    # 링크 및 제목 추출
                    link_element = item.select_one('td.title a')
                    if not link_element:
                        continue
                    
                    relative_url = link_element['href']
                    
                    # 상대 URL 처리
                    if relative_url.startswith('?'):
                        full_url = base_url + "/home/sub.do" + relative_url
                    elif relative_url.startswith('/'):
                        full_url = base_url + relative_url
                    else:
                        full_url = base_url + '/' + relative_url
                    
                    title = link_element.get_text(strip=True)
                    
                    # 작성자(출처) 추출
                    author_element = item.select_one('td.b_write')
                    author = author_element.get_text(strip=True) if author_element else ""
                    
                    # 날짜 추출
                    date_element = item.select_one('td.b_date')
                    pubdate = date_element.get_text(strip=True) if date_element else ""
                    
                    # 조회수 추출 (선택사항)
                    hit_element = item.select_one('td.b_hit')
                    hit_count = hit_element.get_text(strip=True) if hit_element else ""

                    # 본문 추출

                    urls = asyncio.get_event_loop().run_until_complete(extract_preview_urls(full_url))
                    for u in sorted(urls):
                        print(u)

                    # Jupyter에서 asyncio 중첩 실행 허용
                    url = urls[-1]
                    extractor = WebTextExtractor()
                    await extractor.start_browser(headless=True)

                    # text를 변수로 받음
                    text = await extractor.extract_text_from_url(url, clean_text=True)

                    await extractor.close_browser()

                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': text,
                        'author': author,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"오류: 요청 오류 ({current_list_url}): {e}")
                time.sleep(2)
            except Exception as e:
                print(f"오류: 처리 중 오류: {e}")
                traceback.print_exc()
                
        print("\n=== 최종 수집 완료 ===")
        all_articles[name] = articles
    else:
        print('뉴스 기사 목록 사이트:', hwpx_url)
        parsed_url = urllib.parse.urlparse(hwpx_url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
        print(f"Base URL: {base_url}") 
        
        # 2. 헤더를 포함하여 요청 객체를 생성합니다.
        req = urllib.request.Request(hwpx_url, headers=headers)

        # 3. 이 요청으로 URL을 열고, 그 결과를 feedparser로 파싱합니다.
        try:
            with urllib.request.urlopen(req) as response:
                feed = feedparser.parse(response)

                # 정상적으로 파싱되었는지 확인
                if feed.bozo:
                    print("RSS 피드 파싱에 실패했습니다.")
                    # 어떤 에러인지 구체적으로 확인
                    print(f"에러 원인: {feed.bozo_exception}")
                else:
                    print(f"'{feed.feed.title}' RSS 피드에서 가져온 기사 목록입니다.\n")

        except Exception as e:
            print(f"URL을 여는 중 에러가 발생했습니다: {e}")

        for k in range(len(feed.entries)):
            urls = asyncio.get_event_loop().run_until_complete(extract_preview_urls(feed.entries[k]['link']))
            for u in sorted(urls):
                print(u)

            # Jupyter에서 asyncio 중첩 실행 허용
            url = urls[-1]
            extractor = WebTextExtractor()
            await extractor.start_browser(headless=True)

            # text를 변수로 받음
            text = await extractor.extract_text_from_url(url, clean_text=True)

            await extractor.close_browser()
            feed.entries[k]['contents'] = text
        all_articles[name] = feed.entries

식약처
뉴스 기사 목록 사이트: https://www.mfds.go.kr/www/rss/brd.do?brdId=ntc0021
Base URL: https://www.mfds.go.kr
'보도자료' RSS 피드에서 가져온 기사 목록입니다.



[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://www.mfds.go.kr/brd/m_99/view.do?seq=49382                                                    |
✓ | ⏱: 2.56s 

[SCRAPE].. ◆ https://www.mfds.go.kr/brd/m_99/view.do?seq=49382                                                    |
✓ | ⏱: 0.12s 

[COMPLETE] ● https://www.mfds.go.kr/brd/m_99/view.do?seq=49382                                                    |
✓ | ⏱: 2.68s 

...

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://www.kca.go.kr/home/sub.do?menukey=4002&mode=view&no=1003910772&page=1                        |
✓ | ⏱: 1.32s 

[SCRAPE].. ◆ https://www.kca.go.kr/home/sub.do?menukey=4002&mode=view&no=1003910772&page=1                        |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://www.kca.go.kr/home/sub.do?menukey=4002&mode=view&no=1003910772&page=1                        |
✓ | ⏱: 1.37s 

https://www.kca.go.kr/home/synapviewer.do?menukey=4002&fno=10047973&bid=00000013&did=1003910772
https://www.kca.go.kr/home/synapviewer.do?menukey=4002&fno=10047974&bid=00000013&did=1003910772
https://www.kca.go.kr/home/synapviewer.do?menukey=4002&fno=10047975&bid=00000013&did=1003910772
페이지 로딩 중: https://www.kca.go.kr/home/synapviewer.do?menukey=4002&fno=10047975&bid=00000013&did=1003910772
   - [P1-20] 오르테·소베맘 젖병세척기, 전량 자발적 리콜 실시...

=== 최종 수집 완료 ===


In [15]:
all_articles['식약처']

[{'title': '[보도참고] 국민이 선정한 마이나슈 패러디 영상 최우수 작품은?',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': '[보도참고] 국민이 선정한 마이나슈 패러디 영상 최우수 작품은?'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://www.mfds.go.kr/brd/m_99/view.do?seq=49382'}],
  'link': 'https://www.mfds.go.kr/brd/m_99/view.do?seq=49382',
  'content': [{'type': 'text/html',
    'language': None,
    'base': '',
    'value': '[보도참고] 국민이 선정한 마이나슈 패러디 영상 최우수 작품은?'}],
  'summary': '[보도참고] 국민이 선정한 마이나슈 패러디 영상 최우수 작품은?',
  'published': 'Wed, 01 Oct 2025 04:03:04 GMT',
  'published_parsed': time.struct_time(tm_year=2025, tm_mon=10, tm_mday=1, tm_hour=4, tm_min=3, tm_sec=4, tm_wday=2, tm_yday=274, tm_isdst=0),
  'contents': '보도참고자료 보도시점 배포 즉시 배포 2025. 10. 1.(수) <마이너스(-), 나트륨, 슈가’> 국민이 선정한 마이나슈 패러디 영상 최우수 작품은? - ‘덜짜고 덜달게! 건강하게 다시쓰는 명장면’ 패러디 영상 공모전 수상작 7편 공개 - 수상작은 건강한 식생활 문화 확산을 위해 홍보 콘텐츠로 활용 식품의약품안전처(처장 오유경)는 건강한 식생활 실천 문화 확산을 위해 영화·드라마 영상 속의 명장면을 패러디한 ‘마이나슈 패러디 영상 공모

In [14]:
with open('hwpx_output.json', 'w', encoding='utf-8') as outfile:
        json.dump(all_articles, outfile)

# rss sources

In [ ]:
with open('config.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

all_articles = {}
for rss_source in json_data['rss_sources']:
    name = rss_source['name']
    rss_url = rss_source['url']

    # 1. 일반 브라우저인 것처럼 위장할 User-Agent 헤더를 만듭니다.
    #    '명찰'을 달아주는 것과 같습니다.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # 2. 헤더를 포함하여 요청 객체를 생성합니다.
    req = urllib.request.Request(rss_url, headers=headers)

    # 3. 이 요청으로 URL을 열고, 그 결과를 feedparser로 파싱합니다.
    try:
        with urllib.request.urlopen(req) as response:
            feed = feedparser.parse(response)

            # 정상적으로 파싱되었는지 확인
            if feed.bozo:
                print("RSS 피드 파싱에 실패했습니다.")
                # 어떤 에러인지 구체적으로 확인
                print(f"에러 원인: {feed.bozo_exception}")
            else:
                print(f"'{feed.feed.title}' RSS 피드에서 가져온 기사 목록입니다.\n")
                for entry in feed.entries:
                    print(f"제목: {entry.title}")
                    # print(f"카테고리: {entry.section_k} | {entry.section2}")
                    print(f"링크: {entry.link}")
                    print("-" * 20)

    except Exception as e:
        print(f"URL을 여는 중 에러가 발생했습니다: {e}")

    print(f'{len(feed.entries)}개 수집!')
    all_articles[name] = feed.entries
    

'식품음료신문 - 전체기사' RSS 피드에서 가져온 기사 목록입니다.

제목: 오뚜기, “한국 떡 문화의 매력 발산”…K-푸드쇼 ‘떡의 나라’ 10월 공개
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103586
--------------------
제목: ‘K-푸드’ 수출액 최단 기간 100억 달러 돌파
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103584
--------------------
제목: 일동후디스 하이뮨, 100명에게 ‘혼합 선물세트’ 쏜다…신규 TVC 온에어 기념 이벤트
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103583
--------------------
제목: 월드푸드테크협의회-농기평, 푸드테크 산업 글로벌 경쟁력 강화를 위한 업무협약 체결
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103582
--------------------
제목: 푸드테크 미래 ‘푸드테크연구지원센터’ 착공
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103580
--------------------
제목: 전 세계 소비자 눈으로 ‘K-푸드 짝퉁’ 근절한다
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103579
--------------------
제목: 맥도날드, 로코노미 표본 ‘한국의 맛’ 내년부터 확대 시행
링크: http://www.thinkfood.co.kr/news/articleView.html?idxno=103578
--------------------
제목: 편의점 ‘천원 커피’에 저가 커피 시장 흔들
링크: http://www.thinkfood.co.kr/news/article

In [45]:
all_articles['식품안전나라_회수']

[{'title': '주식회사 물맑은고기팜푸드시스템',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': '주식회사 물맑은고기팜푸드시스템'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': '/portal/suspensionDetail.do?search_keyword=3000213663'}],
  'link': '/portal/suspensionDetail.do?search_keyword=3000213663',
  'summary': '한우먹는날 한우 뼈없는 갈비탕',
  'summary_detail': {'type': 'text/html',
   'language': None,
   'base': '',
   'value': '한우먹는날 한우 뼈없는 갈비탕'},
  'published': 'Fri Sep 26 00:00:00 KST 2025',
  'published_parsed': time.struct_time(tm_year=2025, tm_mon=9, tm_mday=26, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=269, tm_isdst=0)},
 {'title': '주식회사 성원에프앤디',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': '주식회사 성원에프앤디'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': '/portal/suspensionDetail.do?search_keyword=3000213600'}],
  'link': '/portal/suspensionDetail.do?search_keyword=3000213600',
 

In [44]:
all_articles.keys()

dict_keys(['식품음료신문', '푸드아이콘', '세이프타임즈', '식약일보', '메디컬투데이', '한국영농신문', '식품안전나라_회수', '식품안전나라_행정처분'])

# scraping sources

In [39]:
with open('extended_config.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

all_articles = {}
for scrape_source in json_data['scraping_sources']:
    name = scrape_source['name']
    list_base_url = scrape_source['url']
    END_PAGE = scrape_source.get('pagenum', 1)
    print('-'*20)
    print(name)
    print('뉴스 기사 목록 사이트:', list_base_url)
    parsed_url = urllib.parse.urlparse(list_base_url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    print(f"Base URL: {base_url}") 
    # 페이지 반복 루프
    START_PAGE = 1
    if name == '식품저널':
        articles = []
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            current_list_url = list_base_url.format(page_num)
            print(f"--- 📚 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            try:
                # 2. 기사 목록 페이지 요청
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                ul_tag = soup.select_one('ul.type1')
                
                # 목록이 없는 경우 (마지막 페이지에 도달한 것으로 간주)
                if not ul_tag:
                    print(f"⚠️ {page_num} 페이지에서 기사 목록을 찾을 수 없습니다. 반복을 종료합니다.")
                    break

                list_items = ul_tag.select('li')
                print(f'✅ {page_num} 페이지에서 기사 {len(list_items)}개 수집')

                # 3. 각 기사 항목 순회 (기존 코드)
                for i, item in enumerate(list_items):
                    keyword = item.select_one('em.info.category').get_text(strip=True)
                    journalist = item.select_one('em.info.name').get_text(strip=True)
                    pubdate = item.select_one('em.info.dated').get_text(strip=True)


                    # A. 제목/URL 추출
                    a_tag = item.select_one('a')
                    if not a_tag:
                        continue

                    title = a_tag.text.strip()
                    relative_url = a_tag['href']
                    full_url = base_url + relative_url

                    # B. 상세 페이지 요청 및 본문 추출
                    contents_response = requests.get(full_url, timeout=10)
                    contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                    article_tag = contents_html.select_one('article#article-view-content-div')
                    
                    # 불필요한 태그 제거 및 텍스트 추출 (좀 더 정교한 방식 권장)
                    if article_tag:
                        contents = article_tag.get_text(strip=True, separator='\n')
                    else:
                        contents = "본문 추출 실패"

                    # C. 결과 저장
                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': contents,
                        'keyword': keyword,
                        'journalist': journalist,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    # 중간 출력
                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')

                    # 서버 부하를 줄이기 위한 지연 시간 추가 (선택 사항이지만 권장)
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 오류 발생 ({current_list_url}): {e}")
                time.sleep(2) # 오류 발생 시 잠시 대기
            except Exception as e:
                print(f"❌ 처리 중 오류 발생: {e}")
                
        print("\n=== ✨ 최종 수집 완료 === ")
        print(f"총 수집된 기사 수: {len(all_articles)}개")
        all_articles[name] = articles
    elif name == '식품안전정보원':
        print(name, '건너뜀')
        continue
        # 페이지 반복 루프
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            current_list_url = list_base_url.format(page_num)
            print(f"--- 📚 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            try:
                # 2. 기사 목록 페이지 요청
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                ul_tag = soup.select_one('ul.type1')
                
                # 목록이 없는 경우 (마지막 페이지에 도달한 것으로 간주)
                if not ul_tag:
                    print(f"⚠️ {page_num} 페이지에서 기사 목록을 찾을 수 없습니다. 반복을 종료합니다.")
                    break

                list_items = ul_tag.select('li')
                print(f'✅ {page_num} 페이지에서 기사 {len(list_items)}개 수집')

                # 3. 각 기사 항목 순회 (기존 코드)
                for i, item in enumerate(list_items):
                    keyword = item.select_one('em.info.category').get_text(strip=True)
                    journalist = item.select_one('em.info.name').get_text(strip=True)
                    pubdate = item.select_one('em.info.dated').get_text(strip=True)


                    # A. 제목/URL 추출
                    a_tag = item.select_one('a')
                    if not a_tag:
                        continue

                    title = a_tag.text.strip()
                    relative_url = a_tag['href']
                    full_url = base_url + relative_url

                    # B. 상세 페이지 요청 및 본문 추출
                    contents_response = requests.get(full_url, timeout=10)
                    contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                    article_tag = contents_html.select_one('article#article-view-content-div')
                    
                    # 불필요한 태그 제거 및 텍스트 추출 (좀 더 정교한 방식 권장)
                    if article_tag:
                        contents = article_tag.get_text(strip=True, separator='\n')
                    else:
                        contents = "본문 추출 실패"

                    # C. 결과 저장
                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': contents,
                        'keyword': keyword,
                        'journalist': journalist,
                        'pubdate': pubdate
                    }
                    articles.extend(article_data)

                    # 중간 출력
                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')

                    # 서버 부하를 줄이기 위한 지연 시간 추가 (선택 사항이지만 권장)
                    time.sleep(0.5)
                    break

            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 오류 발생 ({current_list_url}): {e}")
                time.sleep(2) # 오류 발생 시 잠시 대기
            except Exception as e:
                print(f"❌ 처리 중 오류 발생: {e}")
                
        print("\n=== ✨ 최종 수집 완료 === ")
        print(f"총 수집된 기사 수: {len(all_articles)}개")
        all_articles[name] = articles
        # pprint.pprint(all_articles) # 최종 결과 확인
    elif name == '한국농어민신문':
        articles = [] # 모든 페이지의 기사 데이터를 저장할 리스트
        # 페이지 반복 루프
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            current_list_url = list_base_url.format(page_num)
            print(f"--- 📚 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")
            
            
            # 2. 기사 목록 페이지 요청
            try:
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                ul_tag = soup.select_one('ul.type1')
                # print(ul_tag)
                # 목록이 없는 경우 (마지막 페이지에 도달한 것으로 간주)
                if not ul_tag:
                    print(f"⚠️ {page_num} 페이지에서 기사 목록을 찾을 수 없습니다. 반복을 종료합니다.")
                    break

                list_items = ul_tag.select('li')
                print(f'✅ {page_num} 페이지에서 기사 {len(list_items)}개 수집')

                # 3. 각 기사 항목 순회 (기존 코드)
                for i, item in enumerate(list_items):
                    keyword = item.select_one('em.info.category').get_text(strip=True)
                    journalist = item.select_one('em.info.name').get_text(strip=True)
                    pubdate = item.select_one('em.info.dated').get_text(strip=True)


                    # A. 제목/URL 추출
                    a_tag = item.select_one('a')
                    if not a_tag:
                        continue

                    title = a_tag.text.strip()
                    relative_url = a_tag['href']
                    full_url = base_url + relative_url

                    # B. 상세 페이지 요청 및 본문 추출
                    contents_response = requests.get(full_url, timeout=10)
                    contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                    article_tag = contents_html.select_one('article#article-view-content-div')
                    
                    # 불필요한 태그 제거 및 텍스트 추출 (좀 더 정교한 방식 권장)
                    if article_tag:
                        contents = article_tag.get_text(strip=True, separator='\n')
                    else:
                        contents = "본문 추출 실패"

                    # C. 결과 저장
                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': contents,
                        'keyword': keyword,
                        'journalist': journalist,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    # 중간 출력
                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')

                    # 서버 부하를 줄이기 위한 지연 시간 추가 (선택 사항이지만 권장)
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 오류 발생 ({current_list_url}): {e}")
                time.sleep(2) # 오류 발생 시 잠시 대기
            except Exception as e:
                print(f"❌ 처리 중 오류 발생: {e}")
                
        print("\n=== ✨ 최종 수집 완료 === ")
        all_articles[name] = articles
        # pprint.pprint(all_articles) # 최종 결과 확인
    elif name == '기능식품신문':
        articles = []
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            current_list_url = list_base_url.format((page_num)*10)
            print(f"--- 📚 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")            
            # 2. 기사 목록 페이지 요청
            try:
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                # 뉴스 목록의 메인 테이블을 먼저 찾고, 그 안에서 각 뉴스 항목 찾기
                main_table = soup.select_one('td[style*="padding:8px 10px 8px 10px"] > table')
                
                if not main_table:
                    print(f"경고: {page_num} 페이지에서 기사 목록을 찾을 수 없습니다. 반복을 종료합니다.")
                    break
                
                # 각 뉴스 항목은 tr > td[width="580px"] > table 구조
                news_containers = main_table.select('tr > td[width="580px"]')
                
                # 페이지네이션 등 불필요한 요소 제외 (높이가 30인 tr 제외)
                news_containers = [container for container in news_containers 
                                if container.find('table') and container.find('strong', id='news_title')]

                print(f'완료: {page_num} 페이지에서 기사 {len(news_containers)}개 수집')

                # 3. 각 기사 항목 순회
                for i, container in enumerate(news_containers):
                    # A. 제목/URL/날짜 추출
                    inner_table = container.find('table')
                    title_element = inner_table.select_one('strong#news_title')
                    link_element = inner_table.select_one('a[href*="/news/news.html?mode=view"]')
                    date_element = inner_table.select_one('span#news_date')
                    summary_element = inner_table.select_one('span#news_summary')
                    
                    if not title_element or not link_element:
                        continue

                    title = title_element.get_text(strip=True)
                    relative_url = link_element['href']
                    # HTML 엔티티 디코딩 (&amp; -> &)
                    relative_url = relative_url.replace('&amp;', '&')
                    full_url = base_url + relative_url
                    
                    # 날짜 추출 (대괄호 제거)
                    pubdate = date_element.get_text(strip=True).replace('[', '').replace(']', '') if date_element else ""
                    
                    # 요약 내용 추출 및 정리
                    summary = summary_element.get_text(strip=True) if summary_element else ""

                    # B. 상세 페이지 요청 및 본문 추출
                    try:
                        contents_response = requests.get(full_url, timeout=10)
                        contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                        # 본문 추출 (실제 사이트 구조에 맞게 조정 필요)
                        article_tag = contents_html.select_one('div.article_content, div.content, article')
                        
                        if article_tag:
                            # 불필요한 태그 제거
                            for unwanted in article_tag.select('script, style, .ad, .advertisement'):
                                unwanted.decompose()
                            contents = article_tag.get_text(strip=True, separator='\n')
                        else:
                            contents = summary  # 본문 추출 실패시 요약 사용
                            
                    except Exception as content_error:
                        print(f"   본문 추출 실패: {content_error}")
                        contents = summary

                    # C. 결과 저장
                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'summary': summary,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    # 중간 출력
                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')

                    # 서버 부하를 줄이기 위한 지연 시간 추가
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 오류 발생 ({current_list_url}): {e}")
                time.sleep(2) # 오류 발생 시 잠시 대기
            except Exception as e:
                print(f"❌ 처리 중 오류 발생: {e}")
                
        print("\n=== ✨ 최종 수집 완료 === ")
        all_articles[name] = articles
    elif name == '소비자주권시민회의':
        articles = []
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            current_list_url = list_base_url.format(page_num)
            print(f"--- 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            # 2. 기사 목록 페이지 요청
            try:
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                # 사이트 구조 감지 및 뉴스 항목 추출
                news_items = []
                
                # 첫 번째 사이트 (nutradex) 구조 시도
                main_table = soup.select_one('td[style*="padding:8px 10px 8px 10px"] > table')
                if main_table:
                    print("감지: nutradex 스타일 사이트")
                    containers = main_table.select('tr > td[width="580px"]')
                    news_items = [c for c in containers 
                                if c.find('table') and c.find('strong', id='news_title')]
                
                # 두 번째 사이트 (cucs) 구조 시도
                else:
                    raven_posts = soup.select('div.raven-post-item')
                    if raven_posts:
                        print("감지: WordPress/Raven 스타일 사이트")
                        news_items = raven_posts
                
                # 목록이 없는 경우
                if not news_items:
                    print(f"경고: {page_num} 페이지에서 기사 목록을 찾을 수 없습니다. 반복을 종료합니다.")
                    break

                print(f'완료: {page_num} 페이지에서 기사 {len(news_items)}개 발견')

                # 3. 각 기사 항목 순회
                for i, item in enumerate(news_items):
                    # nutradex 스타일 파싱
                    if item.name == 'td':
                        inner_table = item.find('table')
                        title_element = inner_table.select_one('strong#news_title')
                        link_element = inner_table.select_one('a[href*="/news/news.html?mode=view"]')
                        date_element = inner_table.select_one('span#news_date')
                        summary_element = inner_table.select_one('span#news_summary')
                        
                        if not title_element or not link_element:
                            continue
                        
                        title = title_element.get_text(strip=True)
                        relative_url = link_element['href'].replace('&amp;', '&')
                        full_url = base_url + relative_url
                        pubdate = date_element.get_text(strip=True).replace('[', '').replace(']', '') if date_element else ""
                        summary = summary_element.get_text(strip=True) if summary_element else ""
                    
                    # WordPress/Raven 스타일 파싱
                    elif 'raven-post-item' in item.get('class', []):
                        title_element = item.select_one('h5.raven-post-title a')
                        date_element = item.select_one('a.raven-post-date')
                        summary_element = item.select_one('div.raven-post-excerpt')
                        
                        if not title_element:
                            continue
                        
                        title = title_element.get_text(strip=True)
                        full_url = title_element['href']
                        pubdate = date_element.get_text(strip=True) if date_element else ""
                        summary = summary_element.get_text(strip=True) if summary_element else ""
                    
                    else:
                        continue

                    # B. 상세 페이지 요청 및 본문 추출
                    try:
                        contents_response = requests.get(full_url, timeout=10)
                        contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                        # 다양한 본문 셀렉터 시도
                        article_tag = (
                            contents_html.select_one('div.entry-content') or
                            contents_html.select_one('article.post-content') or
                            contents_html.select_one('div.article_content') or
                            contents_html.select_one('div.content') or
                            contents_html.select_one('article')
                        )
                        
                        if article_tag:
                            # 불필요한 태그 제거
                            for unwanted in article_tag.select('script, style, .ad, .advertisement'):
                                unwanted.decompose()
                            contents = article_tag.get_text(strip=True, separator='\n')
                        else:
                            contents = summary
                            
                    except Exception as content_error:
                        print(f"   본문 추출 실패: {content_error}")
                        contents = summary

                    # C. 결과 저장
                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': contents,
                        'summary': summary,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    # 중간 출력
                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')

                    # 서버 부하를 줄이기 위한 지연 시간
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"오류: 요청 오류 발생 ({current_list_url}): {e}")
                time.sleep(2)
            except Exception as e:
                print(f"오류: 처리 중 오류 발생: {e}")
                import traceback
                traceback.print_exc()
                
        print("\n=== 최종 수집 완료 === ")
        all_articles[name] = articles
        pass
    elif name == '소비자가만드는신문':
        articles = []
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            if '{}' in list_base_url:
                if 'nStart' in list_base_url:
                    # Nutradex: nStart 파라미터 (0, 10, 20...)
                    current_list_url = list_base_url.format((page_num - 1) * 10)
                else:
                    # 일반 페이지 번호
                    current_list_url = list_base_url.format(page_num)
            else:
                current_list_url = list_base_url
                if page_num > 1:
                    print(f"경고: AJAX 페이지네이션. Selenium 필요.")
                    break
            
            print(f"--- 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            try:
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                news_items = []
                site_type = None
                
                # 1. Nutradex 구조
                main_table = soup.select_one('td[style*="padding:8px 10px 8px 10px"] > table')
                if main_table:
                    site_type = "nutradex"
                    print("감지: Nutradex")
                    containers = main_table.select('tr > td[width="580px"]')
                    news_items = [c for c in containers 
                                if c.find('table') and c.find('strong', id='news_title')]
                
                # 2. WordPress/Raven 구조
                elif soup.select('div.raven-post-item'):
                    site_type = "wordpress"
                    print("감지: WordPress/Raven")
                    news_items = soup.select('div.raven-post-item')
                
                # 3. list-block 구조 (소비자고발센터 등)
                elif soup.select('div.list-block'):
                    site_type = "list-block"
                    print("감지: list-block 스타일")
                    news_items = soup.select('div.list-block')
                
                if not news_items:
                    print(f"경고: {page_num} 페이지에서 기사를 찾을 수 없습니다.")
                    break

                print(f'완료: {len(news_items)}개 기사 발견')

                # 각 기사 처리
                for i, item in enumerate(news_items):
                    # Nutradex 파싱
                    if item.name == 'td':
                        inner_table = item.find('table')
                        title_element = inner_table.select_one('strong#news_title')
                        link_element = inner_table.select_one('a[href*="/news/news.html?mode=view"]')
                        date_element = inner_table.select_one('span#news_date')
                        summary_element = inner_table.select_one('span#news_summary')
                        
                        if not title_element or not link_element:
                            continue
                        
                        title = title_element.get_text(strip=True)
                        relative_url = link_element['href'].replace('&amp;', '&')
                        full_url = base_url + relative_url
                        pubdate = date_element.get_text(strip=True).replace('[', '').replace(']', '') if date_element else ""
                        summary = summary_element.get_text(strip=True) if summary_element else ""
                    
                    # WordPress/Raven 파싱
                    elif 'raven-post-item' in item.get('class', []):
                        title_element = item.select_one('h5.raven-post-title a, a.raven-post-title-link')
                        date_element = item.select_one('a.raven-post-date')
                        summary_element = item.select_one('div.raven-post-excerpt')
                        
                        if not title_element:
                            continue
                        
                        title = title_element.get_text(strip=True)
                        full_url = title_element['href']
                        pubdate = date_element.get_text(strip=True) if date_element else ""
                        summary = summary_element.get_text(strip=True) if summary_element else ""
                    
                    # list-block 파싱
                    elif 'list-block' in item.get('class', []):
                        title_element = item.select_one('div.list-titles a strong')
                        link_element = item.select_one('div.list-titles a')
                        date_element = item.select_one('div.list-dated')
                        summary_element = item.select_one('p.list-summary a')
                        
                        if not title_element or not link_element:
                            continue
                        
                        title = title_element.get_text(strip=True)
                        relative_url = link_element['href']
                        
                        # 상대 URL인 경우 base_url 추가
                        if relative_url.startswith('/'):
                            full_url = base_url + relative_url
                        else:
                            full_url = relative_url
                        
                        # 날짜 파싱 (예: "식품 | 정현철 기자 | 2025-09-04 06:10")
                        if date_element:
                            date_text = date_element.get_text(strip=True)
                            parts = date_text.split('|')
                            pubdate = parts[-1].strip() if len(parts) > 0 else date_text
                        else:
                            pubdate = ""
                        
                        category = parts[0].strip() if len(parts) > 0 else date_text
                        summary = summary_element.get_text(strip=True) if summary_element else ""
                    
                    else:
                        continue

                    # 본문 추출
                    try:
                        contents_response = requests.get(full_url, timeout=10)
                        contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                        # 다양한 본문 셀렉터 시도
                        article_tag = (
                            contents_html.select_one('div.user-snp') or  # 소비자고발센터
                            contents_html.select_one('div.entry-content') or
                            contents_html.select_one('article.post-content') or
                            contents_html.select_one('div.article_content') or
                            contents_html.select_one('div.content') or
                            contents_html.select_one('article')
                        )
                        
                        if article_tag:
                            for unwanted in article_tag.select('script, style, .ad, .advertisement'):
                                unwanted.decompose()
                            contents = article_tag.get_text(strip=True, separator='\n')
                        else:
                            contents = summary
                            
                    except Exception as content_error:
                        print(f"   본문 추출 실패: {content_error}")
                        contents = summary

                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'category': category,
                        'summary': summary,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"오류: 요청 오류 ({current_list_url}): {e}")
                time.sleep(2)
            except Exception as e:
                print(f"오류: 처리 중 오류: {e}")
                import traceback
                traceback.print_exc()
                
        print("\n=== 최종 수집 완료 ===")
        all_articles[name] = articles
        pass        
    elif name == '메디파나뉴스':
        articles = []
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            if '{}' in list_base_url:
                current_list_url = list_base_url.format(page_num)
            else:
                current_list_url = list_base_url
            
            print(f"--- 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            try:
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                # 뉴스 항목 선택
                news_list = soup.select('div.lst_feature1 > ul > li')
                
                if not news_list:
                    print(f"경고: {page_num} 페이지에서 기사를 찾을 수 없습니다.")
                    break

                print(f'완료: {len(news_list)}개 기사 발견')

                # 각 기사 처리
                for i, item in enumerate(news_list):
                    # 링크 추출
                    link_element = item.select_one('a')
                    if not link_element:
                        continue
                    
                    relative_url = link_element['href']
                    
                    # 상대 URL 처리
                    if relative_url.startswith('./'):
                        relative_url = relative_url[2:]  # ./ 제거
                    
                    if relative_url.startswith('/'):
                        full_url = base_url + relative_url
                    else:
                        full_url = base_url + '/' + relative_url
                    
                    # 제목 추출
                    title_element = item.select_one('div.tx p.h1')
                    if not title_element:
                        continue
                    title = title_element.get_text(strip=True)
                    
                    # 날짜와 카테고리 추출
                    tit_div = item.select_one('div.tit')
                    category = ""
                    author = ""
                    pubdate = ""
                    
                    if tit_div:
                        tit_paragraphs = tit_div.select('p')
                        if len(tit_paragraphs) >= 3:
                            category = tit_paragraphs[0].get_text(strip=True)  # 예: "유통ㆍ건기식"
                            author = tit_paragraphs[1].get_text(strip=True)     # 예: "최봉선 기자"
                            pubdate = tit_paragraphs[2].get_text(strip=True)   # 예: "25.09.29 09:55"
                    
                    # 요약 추출
                    summary_element = item.select_one('div.tx p.t1')
                    summary = summary_element.get_text(strip=True) if summary_element else ""

                    # 본문 추출
                    try:
                        contents_response = requests.get(full_url, timeout=10)
                        contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                        # 본문 셀렉터 시도
                        article_tag = (
                            contents_html.select_one('div.article_view') or
                            contents_html.select_one('div.content') or
                            contents_html.select_one('article')
                        )
                        
                        if article_tag:
                            for unwanted in article_tag.select('script, style, .ad, .advertisement'):
                                unwanted.decompose()
                            contents = article_tag.get_text(strip=True, separator='\n')
                        else:
                            contents = summary
                            
                    except Exception as content_error:
                        print(f"   본문 추출 실패: {content_error}")
                        contents = summary

                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': contents,
                        'summary': summary,
                        'category': category,
                        'author': author,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"오류: 요청 오류 ({current_list_url}): {e}")
                time.sleep(2)
            except Exception as e:
                print(f"오류: 처리 중 오류: {e}")
                import traceback
                traceback.print_exc()
                
        print("\n=== 최종 수집 완료 ===")
        all_articles[name] = articles
    elif name == '푸드투데이':
        articles = []
        for page_num in range(START_PAGE, END_PAGE + 1):
            # 1. 현재 페이지 URL 생성
            if '{}' in list_base_url:
                current_list_url = list_base_url.format(page_num)
            else:
                current_list_url = list_base_url
            
            print(f"--- 현재 {page_num} 페이지 수집 중 ({current_list_url}) ---")

            try:
                init_response = requests.get(current_list_url, timeout=10)
                soup = BeautifulSoup(init_response.content, 'html.parser')
                
                # 뉴스 항목 선택
                news_list = soup.select('ul.art_list_all > li')
                
                if not news_list:
                    print(f"경고: {page_num} 페이지에서 기사를 찾을 수 없습니다.")
                    break

                print(f'완료: {len(news_list)}개 기사 발견')

                # 각 기사 처리
                for i, item in enumerate(news_list):
                    # 링크 추출
                    link_element = item.select_one('a')
                    if not link_element:
                        continue
                    
                    relative_url = link_element['href']
                    
                    # 상대 URL 처리
                    if relative_url.startswith('/'):
                        full_url = base_url + relative_url
                    else:
                        full_url = base_url + '/' + relative_url
                    
                    # 제목 추출
                    title_element = item.select_one('h2')
                    if not title_element:
                        continue
                    title = title_element.get_text(strip=True)
                    
                    # 요약 추출
                    summary_element = item.select_one('p.ffd')
                    summary = summary_element.get_text(strip=True) if summary_element else ""
                    
                    # 작성자와 날짜 추출
                    author = ""
                    pubdate = ""
                    
                    art_info = item.select('ul.art_info > li')
                    if len(art_info) >= 2:
                        author = art_info[0].get_text(strip=True)   # 예: "황인선 기자"
                        pubdate = art_info[1].get_text(strip=True)  # 예: "2025-09-29 08:55"

                    # 본문 추출
                    try:
                        contents_response = requests.get(full_url, timeout=10)
                        contents_html = BeautifulSoup(contents_response.content, 'html.parser')

                        # 본문 셀렉터 시도
                        article_tag = (
                            contents_html.select_one('div.article_body') or
                            contents_html.select_one('div.art_body') or
                            contents_html.select_one('div.content') or
                            contents_html.select_one('article')
                        )
                        
                        if article_tag:
                            for unwanted in article_tag.select('script, style, .ad, .advertisement'):
                                unwanted.decompose()
                            contents = article_tag.get_text(strip=True, separator='\n')
                        else:
                            contents = summary
                            
                    except Exception as content_error:
                        print(f"   본문 추출 실패: {content_error}")
                        contents = summary

                    article_data = {
                        'page': page_num,
                        'index': i + 1,
                        'title': title,
                        'link': full_url,
                        'content': contents,
                        'summary': summary,
                        'author': author,
                        'pubdate': pubdate
                    }
                    articles.append(article_data)

                    print(f'   - [P{page_num}-{i+1}] {title[:30]}...')
                    time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"오류: 요청 오류 ({current_list_url}): {e}")
                time.sleep(2)
            except Exception as e:
                print(f"오류: 처리 중 오류: {e}")
                import traceback
                traceback.print_exc()
                
        print("\n=== 최종 수집 완료 ===")
        all_articles[name] = articles
        pass
        

--------------------
한국소비자원
뉴스 기사 목록 사이트: https://www.kca.go.kr/home/sub.do?page={}&menukey=4002
Base URL: https://www.kca.go.kr
--------------------
식품저널
뉴스 기사 목록 사이트: https://www.foodnews.co.kr/news/articleList.html?page={}&total=97243&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&box_idxno=&sc_multi_code=&sc_is_image=&sc_is_movie=&sc_user_name=&sc_order_by=E
Base URL: https://www.foodnews.co.kr
--- 📚 현재 1 페이지 수집 중 (https://www.foodnews.co.kr/news/articleList.html?page=1&total=97243&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&box_idxno=&sc_multi_code=&sc_is_image=&sc_is_movie=&sc_user_name=&sc_order_by=E) ---
✅ 1 페이지에서 기사 20개 수집
   - [P1-1] 군산서 식량원조 15만톤 출항ㆍ영양강화립 첫 수출 기념...
   - [P1-2] 이 대통령 “식료품 물가 지나치게 상승…원인부터 꼼꼼히...
   - [P1-3] 오리온, 재생유기농업 활성화 나서...
   - [P1-4] CJ제일제당, 사

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


완료: 20개 기사 발견
   - [P1-1] 아리바이오 탈모 기능성화장품 '텔로액트 (Teloact...
   - [P1-2] 콜마비앤에이치, 글로벌 ESG 평가 '에코바디스 골드'...
   - [P1-3] 마켄, 인천국제공항에 바이오의약품 전용 보세물류센터 오...
   - [P1-4] 대웅제약, 복합 건기식 '합쎈' 와디즈 펀딩 10000...
   - [P1-5] 의약품유통협회, '제14회 사랑.나눔 후원 골프대회' ...
   - [P1-6] 지오영 '인천센터', 리모델링 통해 3PL·4PL '게...
   - [P1-7] JW신약, 佛 듀크레이 '케르티올 컨센트레이트 크림' ...
   - [P1-8] 콜마비앤에이치, 말레이 코스웨이社와 MOU‥동남아 진출...
   - [P1-9] 알피바이오, 제주특산 '덖음귤피' 사업화‥1조 다이어트...
   - [P1-10] 대웅, '에너씨슬 집중샷' 샤인머스켓맛 출시…수험생 라...
   - [P1-11] 현대약품, '미에로화이바 스파클링 제로' 500ml 캔...
   - [P1-12] 대원제약, 약국 전용 건강기능식품 브랜드 '멘토라이프'...
   - [P1-13] 대웅제약, 추석 맞아 '선물 기획전' 진행…최대 78%...
   - [P1-14] 동아제약-국립원예특작과학원, 황해쑥 산업화 확대 MOU...
   - [P1-15] 종근당건강 클리덤, '리포좀 비타민C 라인' 출시...
   - [P1-16] 삼진제약, 관절 건강 ‘콘드로이친 6년 홍삼 진액스틱’...
   - [P1-17] 동국제약, 헤어 클렌징 패드 '마데카 정수리를 닦자' ...
   - [P1-18] 지오영, 의약품 유통업계 최초 'SBTi 온실가스 감축...
   - [P1-19] 알피바이오, '애사비' 생산량 1년 새 10배 급증…올...
   - [P1-20] 일동바사, '비타푸드 아시아' 참가‥포스트바이오틱스 시...

=== 최종 수집 완료 ===
--------------------
푸드투데이
뉴스 기사 목록 사이트: